<a href="https://colab.research.google.com/github/ijazahmad-star/Colab-Workspace/blob/main/AI_Agent_Travel_Planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.7 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.7.0 which is incompatible.


In [ ]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 1.6 MB/s eta 0:00:00


In [ ]:
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 10.6 MB/s eta 0:00:00


In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

In [ ]:
import requests
from pydantic import BaseModel, Field
from typing import List
from langchain.agents import initialize_agent
from langchain.chat_models import init_chat_model
from langchain.memory import ConversationBufferMemory
from langchain.tools import StructuredTool
from langchain.agents.agent_types import AgentType

from langchain.tools import tool
from langchain.schema import SystemMessage


import http.client
import json

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

# api_key = os.getenv("GOOGLE_API_KEY")
# rapid_key = os.getenv("RAPID_API_KEY")
# rapid_host = os.getenv("RAPID_HOST")

from google.colab import userdata

api_key = userdata.get('GOOGLE_API_KEY')
# openai_api_key = userdata.get('OPENAI_API_KEY')
rapid_key = userdata.get('RAPID_API_KEY')
rapid_host = userdata.get('RAPID_HOST')

In [ ]:
flights_data = [
    # --- Domestic Flights (Pakistan) ---
    {"flight_no": "PK301", "origin": "LHE", "destination": "KHI", "date": "2025-10-20", "price": 15000, "airline": "PIA"},
    {"flight_no": "PK306", "origin": "ISB", "destination": "LHE", "date": "2025-10-21", "price": 12000, "airline": "PIA"},
    {"flight_no": "PK372", "origin": "KHI", "destination": "PEW", "date": "2025-10-22", "price": 16500, "airline": "AirSial"},
    {"flight_no": "ER542", "origin": "MUX", "destination": "KHI", "date": "2025-10-23", "price": 14500, "airline": "Serene Air"},
    {"flight_no": "PA201", "origin": "ISB", "destination": "SKZ", "date": "2025-10-24", "price": 13000, "airline": "Airblue"},
    {"flight_no": "NL122", "origin": "KHI", "destination": "LHE", "date": "2025-10-25", "price": 15500, "airline": "AirSial"},
    {"flight_no": "PK607", "origin": "LHE", "destination": "PEW", "date": "2025-10-26", "price": 14000, "airline": "PIA"},
    {"flight_no": "PK350", "origin": "PEW", "destination": "KHI", "date": "2025-10-27", "price": 16000, "airline": "PIA"},
    {"flight_no": "PA214", "origin": "SKZ", "destination": "ISB", "date": "2025-10-28", "price": 12500, "airline": "Airblue"},
    {"flight_no": "ER543", "origin": "KHI", "destination": "MUX", "date": "2025-10-29", "price": 14800, "airline": "Serene Air"},
    {"flight_no": "NL123", "origin": "LHE", "destination": "ISB", "date": "2025-10-30", "price": 11000, "airline": "AirSial"},
    {"flight_no": "PK303", "origin": "KHI", "destination": "LHE", "date": "2025-10-31", "price": 15200, "airline": "PIA"},

    # --- International Flights (From Pakistan) ---
    {"flight_no": "EK622", "origin": "LHE", "destination": "DXB", "date": "2025-10-20", "price": 95000, "airline": "Emirates"},
    {"flight_no": "QR621", "origin": "LHE", "destination": "DOH", "date": "2025-10-21", "price": 78000, "airline": "Qatar Airways"},
    {"flight_no": "SV705", "origin": "KHI", "destination": "JED", "date": "2025-10-22", "price": 82000, "airline": "Saudia"},
    {"flight_no": "EY242", "origin": "ISB", "destination": "AUH", "date": "2025-10-23", "price": 85000, "airline": "Etihad"},
    {"flight_no": "TK715", "origin": "LHE", "destination": "IST", "date": "2025-10-24", "price": 99000, "airline": "Turkish Airlines"},
    {"flight_no": "BA260", "origin": "ISB", "destination": "LHR", "date": "2025-10-25", "price": 210000, "airline": "British Airways"},
    {"flight_no": "EK601", "origin": "KHI", "destination": "DXB", "date": "2025-10-26", "price": 88000, "airline": "Emirates"},
    {"flight_no": "QR610", "origin": "ISB", "destination": "DOH", "date": "2025-10-27", "price": 76000, "airline": "Qatar Airways"},
    {"flight_no": "SV723", "origin": "MUX", "destination": "JED", "date": "2025-10-28", "price": 87000, "airline": "Saudia"},
    {"flight_no": "WY324", "origin": "LHE", "destination": "MCT", "date": "2025-10-29", "price": 74000, "airline": "Oman Air"},
    {"flight_no": "GF765", "origin": "KHI", "destination": "BAH", "date": "2025-10-30", "price": 70000, "airline": "Gulf Air"},
    {"flight_no": "EK625", "origin": "ISB", "destination": "DXB", "date": "2025-10-31", "price": 94000, "airline": "Emirates"},
    {"flight_no": "QR633", "origin": "PEW", "destination": "DOH", "date": "2025-11-01", "price": 77000, "airline": "Qatar Airways"},

    # --- Return Flights (To Pakistan) ---
    {"flight_no": "EK623", "origin": "DXB", "destination": "LHE", "date": "2025-11-05", "price": 97000, "airline": "Emirates"},
    {"flight_no": "QR622", "origin": "DOH", "destination": "LHE", "date": "2025-11-06", "price": 80000, "airline": "Qatar Airways"},
    {"flight_no": "SV706", "origin": "JED", "destination": "KHI", "date": "2025-11-07", "price": 83000, "airline": "Saudia"},
    {"flight_no": "EY243", "origin": "AUH", "destination": "ISB", "date": "2025-11-08", "price": 86000, "airline": "Etihad"},
    {"flight_no": "TK716", "origin": "IST", "destination": "LHE", "date": "2025-11-09", "price": 97000, "airline": "Turkish Airlines"},
    {"flight_no": "BA261", "origin": "LHR", "destination": "ISB", "date": "2025-11-10", "price": 205000, "airline": "British Airways"},
    {"flight_no": "EK602", "origin": "DXB", "destination": "KHI", "date": "2025-11-11", "price": 89000, "airline": "Emirates"},
    {"flight_no": "QR611", "origin": "DOH", "destination": "ISB", "date": "2025-11-12", "price": 75000, "airline": "Qatar Airways"},
    {"flight_no": "SV724", "origin": "JED", "destination": "MUX", "date": "2025-11-13", "price": 88000, "airline": "Saudia"},
    {"flight_no": "WY325", "origin": "MCT", "destination": "LHE", "date": "2025-11-14", "price": 76000, "airline": "Oman Air"},
    {"flight_no": "GF766", "origin": "BAH", "destination": "KHI", "date": "2025-11-15", "price": 72000, "airline": "Gulf Air"},

    # --- International Flights (Abroad to Abroad) ---
    {"flight_no": "EK212", "origin": "IAH", "destination": "DXB", "date": "2025-11-16", "price": 320000, "airline": "Emirates"},
    {"flight_no": "QR706", "origin": "JFK", "destination": "DOH", "date": "2025-11-17", "price": 295000, "airline": "Qatar Airways"},
    {"flight_no": "BA150", "origin": "LHR", "destination": "CDG", "date": "2025-11-18", "price": 55000, "airline": "British Airways"},
    {"flight_no": "LH457", "origin": "LAX", "destination": "FRA", "date": "2025-11-19", "price": 310000, "airline": "Lufthansa"},
    {"flight_no": "AF218", "origin": "CDG", "destination": "DEL", "date": "2025-11-20", "price": 180000, "airline": "Air France"},
    {"flight_no": "SQ323", "origin": "AMS", "destination": "SIN", "date": "2025-11-21", "price": 250000, "airline": "Singapore Airlines"},
    {"flight_no": "CX251", "origin": "HKG", "destination": "LHR", "date": "2025-11-22", "price": 270000, "airline": "Cathay Pacific"},
    {"flight_no": "QR912", "origin": "SYD", "destination": "DOH", "date": "2025-11-23", "price": 315000, "airline": "Qatar Airways"},
    {"flight_no": "EK416", "origin": "DXB", "destination": "SYD", "date": "2025-11-24", "price": 330000, "airline": "Emirates"},
    {"flight_no": "LH779", "origin": "SIN", "destination": "FRA", "date": "2025-11-25", "price": 295000, "airline": "Lufthansa"},
    {"flight_no": "TK1976", "origin": "LHR", "destination": "IST", "date": "2025-11-26", "price": 65000, "airline": "Turkish Airlines"},
    {"flight_no": "AF565", "origin": "NBO", "destination": "CDG", "date": "2025-11-27", "price": 210000, "airline": "Air France"},
    {"flight_no": "BA284", "origin": "SFO", "destination": "LHR", "date": "2025-11-28", "price": 290000, "airline": "British Airways"},
    {"flight_no": "QR728", "origin": "ORD", "destination": "DOH", "date": "2025-11-29", "price": 305000, "airline": "Qatar Airways"},
]

In [ ]:
class FlightSearchInput(BaseModel):
    origin: str = Field(..., description="Origin airport code")
    destination: str = Field(..., description="Destination airport code")
    date: str = Field(..., description="Flight date in YYYY-MM-DD format")

class WeatherInput(BaseModel):
    location: str = Field(..., description="City or region name for weather forecast")

class PoiInput(BaseModel):
    location: str = Field(..., description="City or region name for points of interest")

class HotelInput(BaseModel):
    location: str = Field(..., description="City or region name for hotel search")

@tool(args_schema=HotelInput)
def get_hotel_tool(location: str):
    """
    Get Hotels (name + lat/lon) for a given city.
    """
    limit = 10
    try:
        # Step 1: Search for tourist attractions in the city
        search_query = f"Hotels available in {location}"
        search_url = "https://en.wikipedia.org/w/api.php"
        search_params = {
            "action": "query",
            "list": "search",
            "srsearch": search_query,
            "format": "json",
            "utf8": 1,
            "srlimit": limit
        }

        search_resp = requests.get(search_url, params=search_params, headers={"User-Agent": "vacation-agent"})
        search_data = search_resp.json()

        if "query" not in search_data or not search_data["query"]["search"]:
            return {"error": f"No Hotels found for {location}"}

        # Extract page IDs to fetch coordinates
        page_ids = [str(p["pageid"]) for p in search_data["query"]["search"]]

        # Step 2: Fetch coordinates for all found pages
        coord_url = "https://en.wikipedia.org/w/api.php"
        coord_params = {
            "action": "query",
            "pageids": "|".join(page_ids),
            "prop": "coordinates",
            "format": "json"
        }

        coord_resp = requests.get(coord_url, params=coord_params, headers={"User-Agent": "vacation-agent"})
        coord_data = coord_resp.json()

        # Step 3: Collect results
        results = []
        pages = coord_data.get("query", {}).get("pages", {})

        for page in pages.values():
            title = page.get("title")
            coords = page.get("coordinates", [])
            if coords:
                lat = coords[0].get("lat")
                lon = coords[0].get("lon")
                results.append({
                    "name": title,
                    "latitude": lat,
                    "longitude": lon
                })

        return {"city": location, "locations": results}

    except Exception as e:
        return {"error": str(e)}

@tool(args_schema=PoiInput)
def get_poi_tool(location: str):

    """ Get details about Poi. """

    conn = http.client.HTTPSConnection("opentripmap-places-v1.p.rapidapi.com")

    headers = {
        'x-rapidapi-key': rapid_key,
        'x-rapidapi-host': rapid_host
    }


    conn.request("GET", f"/en/places/geoname?name={location}", headers=headers)
    res = conn.getresponse()
    geo_data = json.loads(res.read().decode("utf-8"))

    lat = geo_data["lat"]
    lon = geo_data["lon"]

    radius = 25000
    conn.request("GET", f"/en/places/radius?radius={radius}&lon={lon}&lat={lat}&rate=3&format=json", headers=headers)
    res = conn.getresponse()
    poi_data = json.loads(res.read().decode("utf-8"))

    # Step 3: Print names of places
    places = []

    for i, place in enumerate(poi_data[:5], start=1):
        places.append(place["name"])
    if places:
        return places
    else:
        return [{"message": "No POI found"}]
@tool(args_schema=FlightSearchInput)
def search_flights_tool(origin: str, destination: str, date: str):
    """ Search flights manually from predefined data. """
    matches = [
        f for f in flights_data
        if f["origin"].lower() == origin.lower()
        and f["destination"].lower() == destination.lower()
        and f["date"] == date.lower()
    ]
    if matches:
        return matches
    else:
        return [{"message": "No flights found"}]

@tool(args_schema=WeatherInput)
def get_weather_tool(location: str):
    """ Get weather forecast for a location. """

    try:
        url = f"https://nominatim.openstreetmap.org/search?format=json&q={location}"
        data = requests.get(url, timeout=10, headers={"User-Agent": "vacation-agent"}).json()
    except Exception as e:
        return {"error": f"failed: {e}"}

    lat, lon = data[0]["lat"], data[0]["lon"]
    try:
        weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
        weather_data = requests.get(weather_url, timeout=10).json()

        return weather_data.get("current_weather", {"error": "Weather data missing"})
    except Exception as e:
        return {"error": f"Weather fetch failed: {e}"}


tools = [search_flights_tool, get_weather_tool, get_poi_tool, get_hotel_tool]

In [ ]:
def search_flights_tool(origin: str, destination: str, date: str):
    matches = [
        f for f in flights_data
        if f["origin"].lower() == origin.lower()
        and f["destination"].lower() == destination.lower()
        and f["date"] == date.lower()
    ]
    if matches:
        return matches
    else:
        return [{"message": "No flights found"}]

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
# llm = ChatOpenAI(
#     model="gpt-4o-mini",
#     temperature=0,
#     api_key=openai_api_key
# )
llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai", api_key = api_key)
# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

system_prompt = SystemMessage(
    content=(
        """
        You are a smart, friendly, and knowledgeable travel assistant.

        Your purpose is to help users plan their trips, explore destinations, and find useful travel information.
        You can access tools to gather real-time or location-specific data such as flight details, local weather, hotels details ,and interesting places to visit.

        When responding:
        - Understand the user's intent clearly (e.g., checking flights, exploring cities, or seeing weather updates).
        - Use your tools only when necessary to provide accurate, current, or location-based information.
        - Present all information in a clear, friendly, and conversational tone — never mention that you are using tools or APIs.
        - Summarize results naturally (e.g., lists of attractions, short flight summaries, or concise weather reports).
        - Avoid technical language, code, or JSON outputs.
        - If information is unavailable, politely inform the user.
        - Keep your responses concise, helpful, and engaging, as if you are a professional travel planner.

        Your overall goal is to make trip planning easier, more enjoyable, and more personalized for the user.

        """
    )
)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    system_message=system_prompt,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    checkpointer=checkpointer,
    verbose=True
)

/tmp/ipython-input-3370092504.py:32: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


## Test Agent

In [ ]:
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"input": [{"role": "user", "content": "i am planning a trip from lahore to peshawar on 26 oct 2025, tell me the weather condition in peshawar?"}]},
    config=config
)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "search_flights_tool",
  "action_input": {
    "origin": "LHE",
    "destination": "PEW",
    "date": "2025-10-26"
  }
}
```
Observation: [{'flight_no': 'PK607', 'origin': 'LHE', 'destination': 'PEW', 'date': '2025-10-26', 'price': 14000, 'airline': 'PIA'}]
Thought:Action:
```
{
  "action": "get_weather_tool",
  "action_input": {
    "location": "Peshawar"
  }
}
```
Observation: {'time': '2025-10-16T06:30', 'interval': 900, 'temperature': 28.8, 'windspeed': 4.9, 'winddirection': 73, 'is_day': 1, 'weathercode': 0}
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "The weather in Peshawar on October 16, 2025, at 06:30 AM is 28.8°C with a wind speed of 4.9 m/s from the east (73 degrees). It is daytime with clear skies."
}
```

> Finished chain.


In [ ]:
print(response)

{'input': [{'role': 'user', 'content': 'i am planning a trip from lahore to peshawar on 26 oct 2025, tell me the weather condition in peshawar?'}], 'output': 'The weather in Peshawar on October 16, 2025, at 06:30 AM is 28.8°C with a wind speed of 4.9 m/s from the east (73 degrees). It is daytime with clear skies.'}


In [ ]:
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"input": [{"role": "user", "content": "what is ask you first about peshawar"}]},
    config=config
)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "I can tell you about the weather, points of interest, and hotels in Peshawar. What would you like to know first?"
}
```

> Finished chain.


In [ ]:
print(response)

{'input': [{'role': 'user', 'content': 'what is ask you first about peshawar'}], 'output': 'I can tell you about the weather, points of interest, and hotels in Peshawar. What would you like to know first?'}


In [ ]:
response = agent.invoke(
    {"input": [{"role": "user", "content": "on what date i ask you for trip?"}]},
    config=config
)

print(response)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "I do not have access to past conversation history, so I don't know on what date you asked about a trip."
}
```

> Finished chain.
{'input': [{'role': 'user', 'content': 'on what date i ask you for trip?'}], 'output': "I do not have access to past conversation history, so I don't know on what date you asked about a trip."}


In [ ]:
response = agent.invoke(
    {"input": "on what date i ask you for trip?"},
    config=config
)

print(response)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "I do not have access to past conversation history, so I cannot tell you the exact date you asked me about a trip."
}
```

> Finished chain.
{'input': 'on what date i ask you for trip?', 'output': 'I do not have access to past conversation history, so I cannot tell you the exact date you asked me about a trip.'}


In [ ]:
while True:
    query = input("User: ")
    if query == "exit":
        break
    response = agent.run(query)
    print(response)

User: i travelling to isb from lhr on 21 oct. sugeest some places to visit and to stay?


> Entering new AgentExecutor chain...
Thought: I need to find points of interest in Islamabad (ISB) and also look for hotels in the same city. I will first search for places to visit in Islamabad.

Action:
```
{
  "action": "get_poi_tool",
  "action_input": {
    "location": "Islamabad"
  }
}
```

Observation: ['Shah Faisal Mosque', 'Faisal Mosque', 'Centaurus Cineplex', 'Islamabad Zoo', 'Daman-e-Koh']
Thought:I have found some points of interest in Islamabad, including Shah Faisal Mosque, Faisal Mosque, Centaurus Cineplex, Islamabad Zoo, and Daman-e-Koh. Now, I will look for hotels in Islamabad for your stay.

Action:
```
{
  "action": "get_hotel_tool",
  "action_input": {
    "location": "Islamabad"
  }
}
```


Observation: {'city': 'Islamabad', 'locations': [{'name': 'Chittagong', 'latitude': 22.335, 'longitude': 91.8325}, {'name': 'Roosevelt Hotel (Manhattan)', 'latitude': 40.75472222, 'longit

In [ ]:
print(memory.buffer)

[HumanMessage(content='i am planing a trip from isb to lahore on 21 oct 2025, book a flight and also give some information about places should i visit?', additional_kwargs={}, response_metadata={}), AIMessage(content='Flight from ISB to LHE on 21 Oct 2025: Flight Number PK306, Airline: PIA, Price: 12,000 PKR. Places to visit: Convent of Jesus and Mary, Lahore Fort, Tomb of Mian Mir.', additional_kwargs={}, response_metadata={}), HumanMessage(content='planning a trip from isb to dubai on 21 oct 2025, suggest some places to stay?', additional_kwargs={}, response_metadata={}), AIMessage(content='Here are some places to stay in Dubai: 1. Atlantis The Royal, Dubai (25.1381, 55.1278) 2. Burj Khalifa area (25.1972, 55.2742) 3. Palm Jumeirah (25.1181, 55.1333) 4. Dubai Maritime City (25.2666, 55.2659).', additional_kwargs={}, response_metadata={}), HumanMessage(content='i travelling to isb from lhr on 21 oct. sugeest some places to visit and to stay?', additional_kwargs={}, response_metadata={